In [4]:
import pandas as pd
import copy
import numpy as np
import pickle
from bertopic import BERTopic

df = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})

/tmp/ipykernel_4280/4031566045.py:7: DtypeWarning: Columns (10,11,15,19,20,21,23,34,35,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_all_labelled.csv", dtype={'year': str})


In [2]:
### chargement du model bertopic

def load_bertopic_model(filename):
    """
    Load a BERTopic model and associated data from a file.
    
    :param filename: The name of the file to load the data from.
    :return: A tuple containing the loaded BERTopic model, topics, probs, and docs variables.
    """
    # Load the BERTopic model
    topic_model = BERTopic.load(filename)
    
    # Load the topics, probs, and docs variables
    with open(filename + '_data.pkl', 'rb') as f:
        topics, probs, embeddings, docs = pickle.load(f)
    
    return topic_model, topics, probs, embeddings, docs

topic_model, topics, probs, embeddings, docs = load_bertopic_model('../models/raw_keybert_bertopic_model')

def create_merged_model(docs, bertopic_model, topics_to_merge_dict, label_names_dict):
    """
    Create a new BERTopic model by merging topics from an existing model.

    This function takes as input a list of documents `docs`, an existing BERTopic model `bertopic_model`, a dictionary `topics_to_merge_dict` specifying which topics to merge, and a dictionary `label_names_dict` specifying the labels for the merged topics.

    The function creates a deep copy of the input BERTopic model and merges the specified topics using the `merge_topics` method. Then, it sets the topic labels for the merged model using the `set_topic_labels` method and the provided `label_names_dict`.

    The resulting merged BERTopic model is then returned.

    Parameters:
        docs (list): A list of documents used to fit the BERTopic model.
        bertopic_model (BERTopic): The input BERTopic model to be merged.
        topics_to_merge_dict (dict): A dictionary specifying which topics to merge. The keys are the topic numbers to be merged, and the values are the topic numbers into which they should be merged.
        label_names_dict (dict): A dictionary specifying the labels for the merged topics. The keys are the topic numbers, and the values are the corresponding labels.

    Returns:
        BERTopic: The resulting merged BERTopic model.
    """
    topic_model_merged = copy.deepcopy(bertopic_model)
    topic_model_merged.merge_topics(docs, topics_to_merge_dict)

    # Create a dictionary to match the aggregated name to their corresponding topic number
    mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names_dict)}
    # Set topic labels for the aggregated model
    topic_model_merged.set_topic_labels(mergedtopic_labels_dict)

    return topic_model_merged### Création du modèle bertopic aggrégé pour topics finaux

# List of topics numbers. Each value of this list is a list that contains the topic number of the topics to join together
topics_to_merge = [ [42,3,0,13], #Delivery Deadlines : challenges and strategies involved in managing delivery deadlines in logistics operations. (vert)
                    [20,50,27], #Quotation and Pricing Strategies (vert bas)
                    [35,32], #Touch Panels and Screens (rouge, haut)
                    [40,36], #Frequency Converters : frequency converters used in industrial applications and the technical support provided by manufacturers and suppliers (rouge, suite)
                    [37,21,6,12,9,4,1,14,16,31,19], #“Automation Components” : hardware and software components used in industrial automation systems. (rouge centre)
                    [33,46,8], #Product Evaluation : evaluate the quality, affordability and reliability of products and services (rouge, fin)
                    [44,51,23,41,49,57,22], #Customer Support : Reliability and Quality in Customer Service and Support (bleu ciel)
                    [58,59], #Quick Customer Service (marron)
                    [38,10,26,52,39,43], #Problem Solving and Communication (focus on the importance of being efficient and precise when solving problems) (jaune)
                    [45,47,55,53,54], #Assistance and Guidance (noir)
                    [29,30,11,24], #Power Supply Issues (2e vert, haut)
                    [7,5,2,25,15,34,18,28,17], #Technical Support (2e vert, bas)
                    [48,56] #None : positive feedback (2e rouge)
]

# Set the topic names for the new aggregated topic
# It must match the order from the topics_to_merge list
label_names = [
    "Outliers",
    "Automation Components",
    "Technical Support",
    "Delivery Deadlines",
    "Problem Solving & Comm",
    "Power Supply Issues",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Product Evaluation",
    "Pricing", #Quotation and Pricing Strategies
    "Assistance", #Assistance and Guidance
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Positive feedback",
    "Quick Customer Service"
    ]

# Create a new merged bertopic model 
topic_model_merged = create_merged_model(docs, topic_model, topics_to_merge, label_names)

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


In [5]:
def get_top_topic_docs(probabilities, df, topic, top_n_docs=len(df)):
    """
    Get the top n documents for a specified topic.

    Parameters:
    probabilities (numpy.ndarray): The probabilities from the BERTopic model.
    df (pandas.DataFrame): The dataframe containing the documents.
    topic (int): The topic to get the top n documents for.
    top_n_docs (int): The number of top documents to return. Defaults to the length of df.

    Returns:
    tuple: A tuple containing a dataframe with the top n documents for the specified topic and a list with their content.
    """
    def sort_docs_with_same_prob(probabilities, prob, topic):
        """
        Sort documents that have the same probability for a specified topic.

        Parameters:
        probabilities (numpy.ndarray): The probabilities from the BERTopic model.
        prob (float): The probability to filter by.
        topic (int): The topic to sort the documents for.

        Returns:
        numpy.ndarray: An array containing the sorted indices of the documents that have the specified probability for the specified topic.
        """
        # Get the indices of all documents that have the current probability for the defined topic
        top_docs_indices = np.where(probabilities[:, topic] == prob)[0]
        # Sort the probabilities for each document in descending order
        sorted_probs = -np.sort(-probabilities[top_docs_indices], axis=1)
        # Compute the score for each document based on the difference between their first and second scores
        scores = sorted_probs[:, 0] - sorted_probs[:, 1]
        # Sort these top documents based on their scores
        sorted_top_docs_indices = top_docs_indices[np.argsort(scores)[::-1]]
        
        return sorted_top_docs_indices
    
    # Get the unique probabilities for the defined topic
    unique_probs = np.unique(probabilities[:, topic])
    # Sort the unique probabilities in descending order
    sorted_unique_probs = np.sort(unique_probs)[::-1]
    
    # Initialize an empty list to store the sorted indices of all documents
    sorted_docs_indices = []
    
    # Loop over the unique probabilities
    for prob in sorted_unique_probs:
        # Sort the documents that have the current probability for the defined topic
        sorted_top_docs_indices = sort_docs_with_same_prob(probabilities, prob, topic)
        # Append these indices to the list of sorted indices of all documents
        sorted_docs_indices.extend(sorted_top_docs_indices)
    
    # Take the top n from this sorted list
    final_top_docs_indices = np.array(sorted_docs_indices)[:top_n_docs]
    
    # Get the content of the top n documents from your dataframe
    top_docs_df = df.iloc[final_top_docs_indices]
    top_docs_content = top_docs_df["processed_data"].to_list()
    
    return top_docs_df, top_docs_content

def filter_docs(df, filter_column, filter_value):
    """
    Filter a dataframe based on a specified column and value.

    Parameters:
    df (pandas.DataFrame): The dataframe to filter.
    filter_column (str): The name of the column to filter by.
    filter_value (str): The value to filter by in the specified column.

    Returns:
    tuple: A tuple containing the filtered dataframe and a boolean mask indicating which rows match the specified filter.
    """
    filter_mask = df[filter_column] == filter_value
    return df[filter_mask], filter_mask

# Filter the top docs based on the specified column and value
filtered_top_docs, filter_mask = filter_docs(df, "Account Country", "France")
top_docs_df, top_docs_content = get_top_topic_docs(topic_model_merged.probabilities_[filter_mask], filtered_top_docs, topic=2, top_n_docs=10)

# top_docs_content